In [8]:
intention = '''Create an easy/safe API to parse/hold/yield constructions of pydantic docx class objects.
In theory this can be generalized and abstracted since the classes are largely absracted. 
However since this is API based, I will just focus on solutions for THIS dictionary task. These can be expanded later, or just have other APIs added
Final objective is have a tool to pass complex commands/instructions, and have it yield a pandas dataframe of the results.
Intermediate steps may be yield a list of lists
I am not yet sure if there is benefit trying to have the whole dictionary in one notional object. 
I don't think so, since each operation is psuedo atomic, and any aggregate actions are intended to be done in something like pandas.
Eventually I would want a loop for pandas operations to easily feed updates to the data. 
'''
conclusion = '''create a pular dictionary specifc datastructure to handle the aggregate paragraphs. 
This will only be per lemma, 
as any aggregations from roots and relations of that sort can be managed in pandas and created by simple inheretance/reference
'''

In [42]:
from typing import Optional, Dict, List, Any, Union, Tuple
import pydantic
from pydantic import ValidationError, validator, root_validator, Field, constr
from pydantic_docx import Docx_Paragraph_and_Runs #type:ignore
import re
import json

In [10]:
outputs = [
   
   'txt_FulaLemmas',
   'txt_FulaRoots',
   
   #extra
   'txt_FulaAnnotations',
   #gloss derivatives
   'txt_FulaSenseEnglish',
   'txt_FulaSenseFrench',
   'txt_FulaSenseClassifications',
   'txt_FulaSenseEnglishAnnotations',
   'txt_FulaSenseFrenchAnnotations',

   #lemma derivative
   'txt_FulaDialects',
   'txt_FulaPOSTags',
   'txt_FulaSynonyms',


   #agg
   'txt_FulaInParenthesis',
   'txt_POS',
   'txt_RootOrigins'
]

In [11]:
noun_patterns = [r'n\.',r"(?<=\()[^\)]+",r"\/"]
nounPatternRegex = re.compile('|'.join([p for p in noun_patterns]))
print(nounPatternRegex)
s = 'n. dfhjkgqh (wsh) askldjhf / asdfhjkkh'
print(re.findall(nounPatternRegex,s))
print(nounPatternRegex.findall(s))

# if /, the next word is a plural. The plural also then usually has a (el) class after it. commas are multiple version
# so each lemma noun may have a (class). The lemma may have one or multiple plurals, which will have a modified (class)
# so have a nouns list? nouns may then be 
# [('n.',{bool}), #only needed if this gets added to all POS? Or maybe theres information in its use?
# ([[{word}],{Optional[class]}]),  #singular
   #([[{word}],{Optional[class]}])]   #plural

# so nounStruct[0][1] is the flag for noun?
   # one POS table, and noun will get added there. But the presence of a noun indicates to look here
# classes may be found [sClass[1] for sClass in nounStruct[1]] will give all classes of singular, [sClass[1] for sClass in nounStruct[1]]

re.compile('n\\.|(?<=\\()[^\\)]+|\\/')
['n.', 'wsh', '/']
['n.', 'wsh', '/']


In [25]:
def getAssert(dct,val,typ:type) -> Any:
   out = dct.get(val,False)
   assert isinstance(out,typ), f"the provided dict {dct} did not give {val} with the expected type: {typ}"
   return out

In [50]:


class dict_entry(pydantic.BaseModel):
   irregularities: List[str] = []
   paragraphs: List[Docx_Paragraph_and_Runs] = Field(...,min_items = 1)
   # lemma: str = Field(...,min_length = 1) #article
   # root: List[Tuple[int,str,str]] #index,root_text, root_meta_data #article
   root_origin: Union[str,bool] = False
   #Paragraph Derivatives: Glosses and Annotations
   englishGloss: Union[str,bool] = Field(...,min_length = 1)
   frenchGloss: Union[str,bool] = Field(...,min_length = 1)
   FulaAnnotations: Union[List[str],bool] #= False #article

   #Gloss Derivatives: senses
   FulaSenseEnglish: Union[List[str],bool] = False #list of "senses" split by semicolons #article
   FulaSenseEnglish_Count: Union[int,bool] = False #number of senses in english #aka FulaSenseClassifications #article
   FulaSenseEnglish_Annotations: Union[List[str],bool] = False #contains the annotations (in parenthesis) for a given sense, if any #article 
   _FulaSenseEnglish_Synonyms: Union[List[str],bool] = False #holds bracket text for a sense, suspected all synonyms. These may all occur at the end, and may be redundant with the synonyms provided at the head of the entry
   _FulaSenseEnglish_Remainder: Union[List[str],bool] = False #holds any run text that is not contained in the above features

   FulaSenseFrench: Union[List[str],bool] = False #article
   FulaSenseFrench_Count: Union[int,bool] = False #aka FulaSenseClassifications
   FulaSenseFrench_Annotations: Union[List[str],bool] = False
   _FulaSenseFrench_Synonyms: Union[List[str],bool] = False
   _FulaSenseFrench_Remainder: Union[List[str],bool] = False
   
   #lemma derivative
   lemmaLine_runs: List[str] #= Field(...)
   # lemmaLine_remainder: List[str] #= Field(...)
   FulaDialects: Union[List[str],bool] = False #article
   FulaPOSTags: Union[List[str],bool] = False #article
   FulaPOSClass: Union[List[str],bool] = False #Noun, Adj, Verb, Adv, Prn, ..., Complicated, Indeterminate
   FulaNoun_NounsAndClass: Union[List[Tuple[str,str]],bool] = False #Pular has unique noun classes. Lemma will be copied here beside its class ("noun", "nounclass"), and any additional singular forms will be in additional tuples in this same list
   FulaNoun_PluralsAndClass: Union[List[Tuple[str,str]],bool] = False #Dict provides plurals for nouns. Tuples will have ("noun", "nounclass")
   FulaSynonyms: Union[List[str],bool] = False #article
   FulaCrossRef: Union[List[str],bool] = False
         # FulaVerbClass:
   
   class Config:
      validate_all = True
      # extra = 'forbid'
      validate_assignment = True
      smart_union = True  

   @root_validator(pre=True)
   def _validate_and_build(cls, values: Dict[str,Any]) -> Dict[str, Any]:
      # print(values)
      newValues: Dict[str,Any] = {}
      newValues['irregularities'] = []
      # newValues[] = getAssert(values,,)
      newValues['paragraphs'] = getAssert(values,'paragraphs',list)
      if len(newValues['paragraphs']) == 0:
         raise ValueError('given invalid paragraphs of zero length')
      else: #logic for subsidary paragraphs
         if len(newValues['paragraphs']) == 1:
            newValues['englishGloss'] = False
            newValues['frenchGloss'] = False
         elif len(newValues['paragraphs']) == 2:
            newValues['irregularities'].append('ambiguous: single subsidary paragraph')
            newValues['englishGloss'] = False
            newValues['frenchGloss'] = False
         elif len(newValues['paragraphs']) == 3:
            newValues['englishGloss'] = True # newValues['paragraphs'][1]
            newValues['frenchGloss'] = True # newValues['paragraphs'][2]
         else: 
            newValues['englishGloss'] = True # newValues['paragraphs'][1]
            newValues['frenchGloss'] = True # newValues['paragraphs'][2]
            newValues['FulaAnnotations'] = True # newValues['paragraphs'][3:]
      

      [print(k,v) for k,v in newValues.items()]
      return newValues

   def parse_senses(self):
      #TODO
      return

   def parse_glossRemainder(self):
      #TODO
      return

   def parse_lemmaLine(self):
      #TODO
      return

   def parse_lemmaLineRemainder(self):
      #TODO
      return

   def get_rootOrigin(self):
      #TODO
      return
   

   def give_entryText(self, joiner = '\t') -> str: #article
      return joiner.join([para.trustyGet('para_text') for para in self.paragraphs])

   def give_root_text(self, main = False, sub = False, both = True) -> str: #article
      if both:
         return ''.join([r[1] for r in self.root])
      if main:
         return  str(self.root[0][1])
      if sub:
         return str(self.root[1][1])
      raise RuntimeError('did not give valid input')

   def trustyGet(self, feat: str, silent_return = True) -> str:
      if not silent_return:
         output : str = getattr(self,feat,'')
         if output:
            return output
         else:
            raise AttributeError(f'trusty getter could not find: {feat}')
      else:
         output : str = getattr(self,feat,'')
         return output
      
            
dict_entry.parse_obj({'paragraphs':[b for a,b in parsed_object_list[5:8]]})

irregularities []
paragraphs [Docx_Paragraph_and_Runs(para_text='your (sg.) (only with certain nouns such as those which refer to close family members)', para_first_line_indent=-91440, run_font_name=['TmsRmn 10pt'], run_bold=[None], run_font_size_pt=[None], paragraph_enumeration=9, run_italic=[None], run_text=['your (sg.) (only with certain nouns such as those which refer to close family members)'], para_left_indent=174193200), Docx_Paragraph_and_Runs(para_text='ton, ta, tes (seulement pour certains substantifs, tels ceux qui se rapportent aux membres de la proche famille)', para_first_line_indent=-91440, run_font_name=['TmsRmn 10pt'], run_bold=[None], run_font_size_pt=[None], paragraph_enumeration=10, run_italic=[None], run_text=['ton, ta, tes (seulement pour certains substantifs, tels ceux qui se rapportent aux membres de la proche famille)'], para_left_indent=174193200), Docx_Paragraph_and_Runs(para_text='aan  prn,ind  DFZH  [an]:Z<->', para_first_line_indent=-91440, run_font_name=[

ValidationError: 5 validation errors for dict_entry
englishGloss
  str type expected (type=type_error.str)
englishGloss
  value could not be parsed to a boolean (type=type_error.bool)
frenchGloss
  str type expected (type=type_error.str)
frenchGloss
  value could not be parsed to a boolean (type=type_error.bool)
FulaAnnotations
  field required (type=value_error.missing)

In [27]:
import pickle
with open('parsed_objectClass_outcomes_dict.pkl', 'rb') as file:
    # Call load method to deserialze
    parsed_objectClass_outcomes_dict = pickle.load(file, encoding='utf-8')

parsed_object_list = parsed_objectClass_outcomes_dict['parsed_object_list'] 
para_text_lookup = parsed_objectClass_outcomes_dict['para_text_lookup'] 
root_ind_list = parsed_objectClass_outcomes_dict['root_ind_list'] 
subroot_ind_list = parsed_objectClass_outcomes_dict['subroot_ind_list'] 
lemma_ind_list = parsed_objectClass_outcomes_dict['lemma_ind_list'] 
root_and_lemma_one_line = parsed_objectClass_outcomes_dict['root_and_lemma_one_line'] 
root_lookup = parsed_objectClass_outcomes_dict['root_lookup'] 
lemma_lookup = parsed_objectClass_outcomes_dict['lemma_lookup'] 
# char_counts = parsed_objectClass_outcomes_dict['char_counts']

In [37]:
# result = []
# for i in parsed_object_list[5:15]:
#    result.append(dict_entry({i))
dict_entry.parse_obj({'paragraphs':parsed_object_list[5:8]})

{'paragraphs': [(9, Docx_Paragraph_and_Runs(para_text='your (sg.) (only with certain nouns such as those which refer to close family members)', para_first_line_indent=-91440, run_font_name=['TmsRmn 10pt'], run_bold=[None], run_font_size_pt=[None], paragraph_enumeration=9, run_italic=[None], run_text=['your (sg.) (only with certain nouns such as those which refer to close family members)'], para_left_indent=174193200)), (10, Docx_Paragraph_and_Runs(para_text='ton, ta, tes (seulement pour certains substantifs, tels ceux qui se rapportent aux membres de la proche famille)', para_first_line_indent=-91440, run_font_name=['TmsRmn 10pt'], run_bold=[None], run_font_size_pt=[None], paragraph_enumeration=10, run_italic=[None], run_text=['ton, ta, tes (seulement pour certains substantifs, tels ceux qui se rapportent aux membres de la proche famille)'], para_left_indent=174193200)), (11, Docx_Paragraph_and_Runs(para_text='aan  prn,ind  DFZH  [an]:Z<->', para_first_line_indent=-91440, run_font_name

dict_entry()